In [1]:
import os
import re
import time
import numpy as np
import rasterio as rio
from osgeo import gdal
import requests
from bs4 import BeautifulSoup
from requests.auth import HTTPBasicAuth
# import dotenv
# dotenv.load_dotenv(dotenv.find_dotenv()) #.env file in the same directory as this script

> This script is intended to download MODIS data from the LP DAAC archive.
> The files to be downloaded are .hdf files.  In this notebook I am trying to download MODIS ET and PET (MOD16A2GF.061). (https://e4ftl01.cr.usgs.gov/MOLT/MOD16A2GF.061/)
> Each hdf file contains both of these variables and the files are in timestamped folders.
> To download the data, one needs an earthdata login so I have provided my credentials to authorize my access
> The Problem is the code can find the files but I cant download them. I have confirmed that the my login is correct.

This code needs to be fixed so that it works!!! It does not download any files!!!

> Reference: https://github.com/bpostance/learn_data_engineering/blob/main/earth.observation/modis/MCD64A1v061-burned-areas/00.ETL-MODIS.ipynb

Earth data login credentials

In [14]:
# params fo LP DAAC
# you will require an EarthData login
host = fr'https://e4ftl01.cr.usgs.gov/MOLT/MOD16A2GF.061'
login = 'Katoria'
password = 'Firstclass0902'

outDir = "D:/tmp/KaToRiA"

In [15]:
# list folders from which to download
r = requests.get(host, verify=True, stream=True,auth=(login,password))
soup = BeautifulSoup(r.text, "html.parser")
folders = list()
for link in soup.findAll('a', attrs={'href': re.compile("\d{4}.\d{2}.\d{2}/")}): #e.g. 2000.03.05/
    folders.append(link.get('href'))
print(f"{len(folders)} folders found")

1104 folders found


In [16]:
# list files in folders
for f in folders[:1]:
    file_list = list()
    folder_url = f"{host}/{f}"
    r = requests.get(folder_url, verify=True, stream=True,auth=(login,password))
    soup = BeautifulSoup(r.text, "html.parser")
    for link in soup.findAll('a', attrs={'href': re.compile(".hdf$")}):
        file_list.append(link.get('href'))    
print(f"{len(file_list)} files found in folder n")

283 files found in folder n


In [17]:
# Since MODIS data is stored in numbered grids, select tiles corresponding to areas of interest.
#Kenya lies in 4 grids: h21v08, h22v08, h21v09, h22v09 so select only these tiles for download.
# https://modis-land.gsfc.nasa.gov/MODLAND_grid.html
hreg = re.compile("h21|h22") # Kenya is in h21 and h22
vreg = re.compile("v0[8-9]") # Kenya is in v08 and v09
ken_files = list()
for fl in file_list:
    h = hreg.search(fl)
    if h:
        v = vreg.search(h.string)
        if v:
            ken_files.append(v.string)
print(ken_files)
print(f"{len(ken_files)} KENYA tiles found")

['MOD16A2GF.A2000001.h21v08.061.2020256104621.hdf', 'MOD16A2GF.A2000001.h21v09.061.2020256103437.hdf', 'MOD16A2GF.A2000001.h22v08.061.2020256103529.hdf', 'MOD16A2GF.A2000001.h22v09.061.2020256103517.hdf']
4 KENYA tiles found


#### The code works fine (or so I think up to here)
> In the next cell it fails to download the files

In [27]:
class SessionWithHeaderRedirection(requests.Session):
    AUTH_HOST = 'urs.earthdata.nasa.gov'
    
    def __init__(self, username, password):
        super().__init__()
        self.auth = (username, password)

    def rebuild_auth(self, prepared_request, response):
        headers = prepared_request.headers
        url = prepared_request.url
        if 'Authorization' in headers:
            original_parsed = requests.utils.urlparse(response.request.url)
            redirect_parsed = requests.utils.urlparse(url)
            if (original_parsed.hostname != redirect_parsed.hostname) and \
                    redirect_parsed.hostname != self.AUTH_HOST and \
                    original_parsed.hostname != self.AUTH_HOST:
                del headers['Authorization']


session = SessionWithHeaderRedirection(login, password)

for f in folders[:1]:
    for fl in ken_files[:]:
        try:
            file_url = f"{host}/{f}{fl}"
            response = session.get(file_url, stream=True)

            if response.ok:
                if not os.path.exists(f"{outDir}"): os.makedirs(f"{outDir}")
                print('Request successful, downloading to ' + f"{outDir}/{fl}")
                with open(f"{outDir}/{fl}", 'wb') as fL:
                    for chunk in response.iter_content(chunk_size=1024*1024): fL.write(chunk)
                
                print(f"Downloaded {file_url}")
                time.sleep(4) # be nice to the server
            elif response.status_code == 404:
                print(f"File not found: {file_url}")
            elif response.status_code == 401:
                print(f"Unauthorized access: {file_url}")
            else:
                print('Request failed: ', response.text)

        except: print(f"Muhahahahahahaha! Error!: {fl}")


2000.01.01/
Request successful, downloading to D:/tmp/KaToRiA/MOD16A2GF.A2000001.h21v08.061.2020256104621.hdf
Downloaded https://e4ftl01.cr.usgs.gov/MOLT/MOD16A2GF.061/2000.01.01/MOD16A2GF.A2000001.h21v08.061.2020256104621.hdf
2000.01.01/
Request successful, downloading to D:/tmp/KaToRiA/MOD16A2GF.A2000001.h21v09.061.2020256103437.hdf
Downloaded https://e4ftl01.cr.usgs.gov/MOLT/MOD16A2GF.061/2000.01.01/MOD16A2GF.A2000001.h21v09.061.2020256103437.hdf
2000.01.01/
Request successful, downloading to D:/tmp/KaToRiA/MOD16A2GF.A2000001.h22v08.061.2020256103529.hdf
Downloaded https://e4ftl01.cr.usgs.gov/MOLT/MOD16A2GF.061/2000.01.01/MOD16A2GF.A2000001.h22v08.061.2020256103529.hdf
2000.01.01/
Request successful, downloading to D:/tmp/KaToRiA/MOD16A2GF.A2000001.h22v09.061.2020256103517.hdf
Downloaded https://e4ftl01.cr.usgs.gov/MOLT/MOD16A2GF.061/2000.01.01/MOD16A2GF.A2000001.h22v09.061.2020256103517.hdf
